In [18]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [19]:
# Read in cleaned movies DataFrame
movies = pd.read_pickle('data/movies_reduced.pkl')

In [20]:
all_genres = []
for i, row in movies.iterrows():
    for genre in row.genres_list:
        if genre not in all_genres:
            all_genres.append(genre)

In [62]:
# 80% train, 20% test (41870 total with a genre label)
# Splitting later with keras
genre_all = movies[movies.num_genres!=0]

In [63]:
texts = []
labels = []
for i,row in genre_all.iterrows():
    for g in row.genres_list:
        labels.append(g)
        texts.append(row.overview)

In [64]:
dict_labels = list(set(labels))
dict_values = list(range(0,len(dict_labels)))
labels_index = dict(zip(dict_labels,dict_values))

In [65]:
labels = [labels_index[l] for l in labels]

In [70]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2

In [67]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [68]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 83298 unique tokens.
Shape of data tensor: (89120, 1000)
Shape of label tensor: (89120, 20)


In [71]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [79]:
embeddings_index = {}
f = open('TextEmbedding/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
EMBEDDING_DIM = 100

Found 400000 word vectors.


In [80]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [81]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [89]:
from keras.engine.topology import Input
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation,Conv1D,Flatten
from keras.models import Sequential,Modelmodel.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=128)

In [90]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

In [91]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [92]:
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=128)

Train on 71296 samples, validate on 17824 samples
Epoch 1/2
71296/71296 [==============================] - 1628s 23ms/step - loss: 2.3511 - acc: 0.2564 - val_loss: 2.1727 - val_acc: 0.2907
Epoch 2/2
71296/71296 [==============================] - 1724s 24ms/step - loss: 2.1336 - acc: 0.2946 - val_loss: 2.1141 - val_acc: 0.2952


### Keras - RNN

In [ ]:
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
genre_colNames = ['genre_'+ g for g in all_genres]

In [ ]:
labels_matrix = movies[genre_colNames]
print(labels_matrix.shape)
print(len(movies.overview))

In [ ]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(movies.overview)
sequences = tokenizer.texts_to_sequences(movies.overview)
train_data = pad_sequences(sequences,maxlen=1000)

In [ ]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=1000))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(20, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_data, labels_matrix, validation_split=0.8, epochs=3)

In [ ]:
tokenizer_test = Tokenizer(num_words=1000)
tokenizer_test.fit_on_texts(nb_genre_test.overview)
sequences_test = tokenizer_test.texts_to_sequences(nb_genre_test.overview)
test_data = pad_sequences(sequences_test,maxlen=1000)

In [ ]:
nn_preds = model.predict(test_data)

In [ ]:
nn_preds.shape